In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from resnet1d import Res34SimSiam, Res34SimSiamSplitHeads, Resnet34Baseline
from dataset import Dataset_per_file, Dataset_whole, Dataset_ori
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torch.autograd import Variable
import os
import numpy as np
from tqdm import tqdm
import argparse
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc
import sys
import gc
from collections import OrderedDict


In [13]:
def test_epoch(PPG_model, test_loader, baseline=False):
    with torch.no_grad():
       

        PPG_preds = None
        PPG_pred_probs = None
        all_targets = None
        
        PPG_model.eval()
        tstart = datetime.now()

        for batch_idx, (PPG, target) in enumerate(test_loader):

            PPG = PPG.cuda().float()
            target = target.cuda().long()
            
            if baseline:
                _, PPG_out = PPG_model(PPG, PPG)
            else:
                _, _, _, _, _, PPG_out = PPG_model(PPG, PPG)
            
            PPG_predicted = PPG_out.argmax(1)
            PPG_predicted_prob = F.softmax(PPG_out, dim=1)[:, 1]

            if PPG_preds == None:
                PPG_pred_probs = PPG_predicted_prob
                PPG_preds = PPG_predicted
                all_targets = target
            else:
                PPG_preds = torch.cat((PPG_preds, PPG_predicted))
                PPG_pred_probs = torch.cat((PPG_pred_probs, PPG_predicted_prob))
                all_targets = torch.cat((all_targets, target))
        tend = datetime.now()


        precision, recall, thresholds = precision_recall_curve(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
        pr_auc = auc(recall, precision)

        print(f'[TEST] \tPPG      F1: {round(f1_score(all_targets.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG ROC AUC: {round(roc_auc_score(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG PR  AUC: {round(pr_auc, 4)}')

        
def run_test_on_model(MODEL_PATH):
    
    model = Res34SimSiam(512, 128).cuda()

    state_dict = torch.load(MODEL_PATH) 
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] #remove 'module'
        new_state_dict[name] = v

    state_dict = new_state_dict
    model.load_state_dict(state_dict)

    testing_data_label_pairs = [
        ['data_simband_ecg_2400.npy', 'label_simband.npy'],
        ['data_simband_ppg_2400.npy', 'label_simband.npy'],
        ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy'],
        ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy'],
        ['data_staford_2400.npy', 'label_staford.npy'],
        ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy'],
        ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy']
    ]
    test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
    
    for pair in testing_data_label_pairs:

        print(pair[0].split('.')[0].split('/')[-1])

        x_path = test_data_folder + pair[0]
        y_path = test_data_folder + pair[1]

#         print(x_path)
#         print(y_path)
        test_dataset = Dataset_ori(x_path, y_path)
        testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

        test_epoch(model, testloader)

        print()
        
    del model
    gc.collect()

# 0.2

In [14]:
run_test_on_model('/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_0.2_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt')

data_simband_ecg_2400
[TEST] 	PPG      F1: 0.6221
[TEST] 	PPG ROC AUC: 0.7629
[TEST] 	PPG PR  AUC: 0.7327

data_simband_ppg_2400
[TEST] 	PPG      F1: 0.7703
[TEST] 	PPG ROC AUC: 0.9185
[TEST] 	PPG PR  AUC: 0.8741

data_ucla_ecg_2400
[TEST] 	PPG      F1: 0.7741
[TEST] 	PPG ROC AUC: 0.9138
[TEST] 	PPG PR  AUC: 0.7537

data_ucla_ppg_2400
[TEST] 	PPG      F1: 0.6946
[TEST] 	PPG ROC AUC: 0.9254
[TEST] 	PPG PR  AUC: 0.7393

data_staford_2400
[TEST] 	PPG      F1: 0.5875
[TEST] 	PPG ROC AUC: 0.8552
[TEST] 	PPG PR  AUC: 0.6706

data_staford_goodquality_2400
[TEST] 	PPG      F1: 0.967
[TEST] 	PPG ROC AUC: 0.9997
[TEST] 	PPG PR  AUC: 0.9978

data_staford_badquality_2400
[TEST] 	PPG      F1: 0.5531
[TEST] 	PPG ROC AUC: 0.792
[TEST] 	PPG PR  AUC: 0.5989



# 0.4

In [15]:
run_test_on_model('/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_0.4_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt')

data_simband_ecg_2400
[TEST] 	PPG      F1: 0.6406
[TEST] 	PPG ROC AUC: 0.7628
[TEST] 	PPG PR  AUC: 0.7401

data_simband_ppg_2400
[TEST] 	PPG      F1: 0.7607
[TEST] 	PPG ROC AUC: 0.9092
[TEST] 	PPG PR  AUC: 0.8546

data_ucla_ecg_2400
[TEST] 	PPG      F1: 0.7639
[TEST] 	PPG ROC AUC: 0.9097
[TEST] 	PPG PR  AUC: 0.7512

data_ucla_ppg_2400
[TEST] 	PPG      F1: 0.6916
[TEST] 	PPG ROC AUC: 0.9248
[TEST] 	PPG PR  AUC: 0.7356

data_staford_2400
[TEST] 	PPG      F1: 0.5773
[TEST] 	PPG ROC AUC: 0.853
[TEST] 	PPG PR  AUC: 0.6779

data_staford_goodquality_2400
[TEST] 	PPG      F1: 0.9448
[TEST] 	PPG ROC AUC: 0.9994
[TEST] 	PPG PR  AUC: 0.9964

data_staford_badquality_2400
[TEST] 	PPG      F1: 0.547
[TEST] 	PPG ROC AUC: 0.7881
[TEST] 	PPG PR  AUC: 0.6064



# 0.6

In [16]:
run_test_on_model('/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_0.6_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt')

data_simband_ecg_2400
[TEST] 	PPG      F1: 0.6498
[TEST] 	PPG ROC AUC: 0.7715
[TEST] 	PPG PR  AUC: 0.7532

data_simband_ppg_2400
[TEST] 	PPG      F1: 0.7762
[TEST] 	PPG ROC AUC: 0.9208
[TEST] 	PPG PR  AUC: 0.8754

data_ucla_ecg_2400
[TEST] 	PPG      F1: 0.761
[TEST] 	PPG ROC AUC: 0.9125
[TEST] 	PPG PR  AUC: 0.7951

data_ucla_ppg_2400
[TEST] 	PPG      F1: 0.6956
[TEST] 	PPG ROC AUC: 0.9239
[TEST] 	PPG PR  AUC: 0.7971

data_staford_2400
[TEST] 	PPG      F1: 0.5894
[TEST] 	PPG ROC AUC: 0.863
[TEST] 	PPG PR  AUC: 0.7046

data_staford_goodquality_2400
[TEST] 	PPG      F1: 0.9619
[TEST] 	PPG ROC AUC: 0.9996
[TEST] 	PPG PR  AUC: 0.9981

data_staford_badquality_2400
[TEST] 	PPG      F1: 0.558
[TEST] 	PPG ROC AUC: 0.8011
[TEST] 	PPG PR  AUC: 0.636



# 0.8

In [17]:
run_test_on_model('/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_0.8_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt')

data_simband_ecg_2400
[TEST] 	PPG      F1: 0.594
[TEST] 	PPG ROC AUC: 0.7673
[TEST] 	PPG PR  AUC: 0.7447

data_simband_ppg_2400
[TEST] 	PPG      F1: 0.7544
[TEST] 	PPG ROC AUC: 0.9194
[TEST] 	PPG PR  AUC: 0.8752

data_ucla_ecg_2400
[TEST] 	PPG      F1: 0.7667
[TEST] 	PPG ROC AUC: 0.9196
[TEST] 	PPG PR  AUC: 0.7734

data_ucla_ppg_2400
[TEST] 	PPG      F1: 0.6983
[TEST] 	PPG ROC AUC: 0.9251
[TEST] 	PPG PR  AUC: 0.7626

data_staford_2400
[TEST] 	PPG      F1: 0.5794
[TEST] 	PPG ROC AUC: 0.8589
[TEST] 	PPG PR  AUC: 0.7083

data_staford_goodquality_2400
[TEST] 	PPG      F1: 0.9783
[TEST] 	PPG ROC AUC: 0.998
[TEST] 	PPG PR  AUC: 0.9955

data_staford_badquality_2400
[TEST] 	PPG      F1: 0.5448
[TEST] 	PPG ROC AUC: 0.7964
[TEST] 	PPG PR  AUC: 0.6438



# 1.2

In [18]:
run_test_on_model('/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.2_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt')

data_simband_ecg_2400
[TEST] 	PPG      F1: 0.6423
[TEST] 	PPG ROC AUC: 0.7733
[TEST] 	PPG PR  AUC: 0.7577

data_simband_ppg_2400
[TEST] 	PPG      F1: 0.7773
[TEST] 	PPG ROC AUC: 0.9167
[TEST] 	PPG PR  AUC: 0.8713

data_ucla_ecg_2400
[TEST] 	PPG      F1: 0.7784
[TEST] 	PPG ROC AUC: 0.9037
[TEST] 	PPG PR  AUC: 0.8021

data_ucla_ppg_2400
[TEST] 	PPG      F1: 0.7189
[TEST] 	PPG ROC AUC: 0.9284
[TEST] 	PPG PR  AUC: 0.7926

data_staford_2400
[TEST] 	PPG      F1: 0.5948
[TEST] 	PPG ROC AUC: 0.8694
[TEST] 	PPG PR  AUC: 0.7031

data_staford_goodquality_2400
[TEST] 	PPG      F1: 0.973
[TEST] 	PPG ROC AUC: 0.9995
[TEST] 	PPG PR  AUC: 0.997

data_staford_badquality_2400
[TEST] 	PPG      F1: 0.5639
[TEST] 	PPG ROC AUC: 0.8115
[TEST] 	PPG PR  AUC: 0.6386

